In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import json

In [5]:
! conda install -c conda-forge folium --yes
import folium # plotting library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.8.3-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.60 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.62 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.60 MB/s
folium-0.8.3-p 100% |################################| Time: 0:00:00  51.90 MB/s


### Input the foursquare API credentials

In [6]:
CLIENT_ID = 'IRQWVERHQVTMY01WM1LYRVPGTIWFS5JPAU153FXNLO2NIMCU' # your Foursquare ID
CLIENT_SECRET = 'L41VS3XBRFYHUJ5XNH3QEQW1DGEXMCQKXLFSKJFDYMP0DHJL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IRQWVERHQVTMY01WM1LYRVPGTIWFS5JPAU153FXNLO2NIMCU
CLIENT_SECRET:L41VS3XBRFYHUJ5XNH3QEQW1DGEXMCQKXLFSKJFDYMP0DHJL


### Choose downtown San Francisco as the starting point of our search
### This location here is a very popular Chinese restaurant near SF China town

In [26]:
address = '631 Kearny St, San Francisco, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.79413 -122.404838


### Since we are helping a Chinese restaurant owner, let's search for some Chinese restaurant nearby

In [27]:
search_query = 'Chinese'
radius = 500
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id=CLIENT_ID,
  client_secret=CLIENT_SECRET,
  v='20180323',
  ll='37.79413,-122.404838',
  query=search_query,
  radius=500,
  limit=60
)
resp = requests.get(url=url, params=params)
results = json.loads(resp.text)

### This is what the returned result looks like

In [28]:
results

{'meta': {'code': 200, 'requestId': '5c969c044434b961185bb54e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-56f2092e498eac8f5f0799c8-0',
      'venue': {'beenHere': {'count': 0,
        'lastCheckinExpiredAt': 0,
        'marked': False,
        'unconfirmedCount': 0},
       'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d145941735',
         'name': 'Chinese Restaurant',
         'pluralName': 'Chinese Restaurants',
         'primary': True,
         'shortName': 'Chinese'}],
       'contact': {},
       'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
       'id': '56f2092e498eac8f5f0799c8',
       'location': {'address': '28 Waverly Pl.',
        'cc': 'US',
        'city': 'San Francisco',
  

### Convert the json file to dataframe

In [29]:
# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.describe()

,reasons.count,venue.beenHere.count,venue.beenHere.lastCheckinExpiredAt,venue.beenHere.unconfirmedCount,venue.hereNow.count,venue.location.distance,venue.location.lat,venue.location.lng,venue.photos.count,venue.stats.checkinsCount,venue.stats.tipCount,venue.stats.usersCount,venue.stats.visitsCount
count,60.0,60.0,60.0,60.0,60.000000,60.000000,60.000000,60.000000,60.0,60.0,60.0,60.0,60.0
mean,0.0,0.0,0.0,0.0,0.116667,220.333333,37.795090,-122.406047,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.323732,117.267862,0.001488,0.001286,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.000000,11.000000,37.790922,-122.408774,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.000000,148.500000,37.794105,-122.406761,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.000000,224.000000,37.795058,-122.406092,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.000000,268.250000,37.796149,-122.405107,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,1.000000,499.000000,37.798265,-122.403065,0.0,0.0,0.0,0.0,0.0


### Only keep the restaurant name and geo position

### The tips, num of visits etc data from the foursquare API are all 0, so we are not using them, otherwise it would make our analysis more informative

In [30]:
nearby_res=dataframe[['venue.name','venue.location.lat','venue.location.lng']]
nearby_res.columns = ['Restaurant', 'Latitude','Longitude']
nearby_res.head()

,Restaurant,Latitude,Longitude
0,Mister Jiu's,37.793790,-122.406615
1,Hunan Home's Restaurant,37.796193,-122.405581
2,Hong Kong Clay Pot Restaurant,37.795844,-122.406594
3,City View Restaurant,37.794191,-122.403918
4,R&G Lounge 嶺南小館,37.794072,-122.404724


### Do K-Means clustering analysis on the Latitude and Longitude of the restaurants
### So that we can have an idea of the geographical distribution of these Chinese restaurants

In [31]:
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# set number of clusters
kclusters = 5

nearby_res_clustering = nearby_res.drop('Restaurant', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

nearby_res['Cluster Label']=kmeans.labels_
nearby_res.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Restaurant,Latitude,Longitude,Cluster Label
0,Mister Jiu's,37.793790,-122.406615,0
1,Hunan Home's Restaurant,37.796193,-122.405581,1
2,Hong Kong Clay Pot Restaurant,37.795844,-122.406594,1
3,City View Restaurant,37.794191,-122.403918,3
4,R&G Lounge 嶺南小館,37.794072,-122.404724,3


In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_res['Latitude'], nearby_res['Longitude'], nearby_res['Restaurant'], nearby_res['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
#map_clusters

### Let's try a different area, in this case, the Nob Hill area

In [35]:
address = '1325 Leavenworth St, San Francisco, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
lat_lng=str(latitude)+','+str(longitude)

search_query = 'Chinese'
radius = 500
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id=CLIENT_ID,
  client_secret=CLIENT_SECRET,
  v='20180323',
  ll=lat_lng,
  query=search_query,
  radius=500,
  limit=60
)
resp = requests.get(url=url, params=params)
results = json.loads(resp.text)

# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

nearby_res=dataframe[['venue.name','venue.location.lat','venue.location.lng']]
nearby_res.columns = ['Restaurant', 'Latitude','Longitude']

from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# set number of clusters
kclusters = 2

nearby_res_clustering = nearby_res.drop('Restaurant', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

nearby_res['Cluster Label']=kmeans.labels_

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_res['Latitude'], nearby_res['Longitude'], nearby_res['Restaurant'], nearby_res['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
#map_clusters

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Try Sunset district next

In [37]:
address = '3010 Noriega St, San Francisco, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
lat_lng=str(latitude)+','+str(longitude)

search_query = 'Chinese'
radius = 500
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id=CLIENT_ID,
  client_secret=CLIENT_SECRET,
  v='20180323',
  ll=lat_lng,
  query=search_query,
  radius=1000,
  limit=60
)
resp = requests.get(url=url, params=params)
results = json.loads(resp.text)

# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

nearby_res=dataframe[['venue.name','venue.location.lat','venue.location.lng']]
nearby_res.columns = ['Restaurant', 'Latitude','Longitude']

from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# set number of clusters
kclusters = 2

nearby_res_clustering = nearby_res.drop('Restaurant', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

nearby_res['Cluster Label']=kmeans.labels_

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_res['Latitude'], nearby_res['Longitude'], nearby_res['Restaurant'], nearby_res['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
#map_clusters

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### let's try Richmond District

In [39]:
address = '6210 Geary Blvd, San Francisco, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
lat_lng=str(latitude)+','+str(longitude)

search_query = 'Chinese'
radius = 500
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id=CLIENT_ID,
  client_secret=CLIENT_SECRET,
  v='20180323',
  ll=lat_lng,
  query=search_query,
  radius=1000,
  limit=60
)
resp = requests.get(url=url, params=params)
results = json.loads(resp.text)

# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

nearby_res=dataframe[['venue.name','venue.location.lat','venue.location.lng']]
nearby_res.columns = ['Restaurant', 'Latitude','Longitude']

from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# set number of clusters
kclusters = 3

nearby_res_clustering = nearby_res.drop('Restaurant', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

nearby_res['Cluster Label']=kmeans.labels_

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_res['Latitude'], nearby_res['Longitude'], nearby_res['Restaurant'], nearby_res['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
#map_clusters

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### let's try Fisherman's Wharf Area

In [42]:
address = '2500 Mason St, San Francisco, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
lat_lng=str(latitude)+','+str(longitude)

search_query = 'Chinese'
radius = 500
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
  client_id=CLIENT_ID,
  client_secret=CLIENT_SECRET,
  v='20180323',
  ll=lat_lng,
  query=search_query,
  radius=1000,
  limit=60
)
resp = requests.get(url=url, params=params)
results = json.loads(resp.text)

# assign relevant part of JSON to venues
venues = results['response']['groups'][0]['items']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

nearby_res=dataframe[['venue.name','venue.location.lat','venue.location.lng']]
nearby_res.columns = ['Restaurant', 'Latitude','Longitude']

from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# set number of clusters
kclusters = 3

nearby_res_clustering = nearby_res.drop('Restaurant', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

nearby_res['Cluster Label']=kmeans.labels_

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_res['Latitude'], nearby_res['Longitude'], nearby_res['Restaurant'], nearby_res['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
